In [1]:
import tensorflow as tf
import numpy as np

In [2]:
with np.load("notMNIST.npz") as data:
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [3]:
def linear(x, numHidden):
    numInput = int(x.get_shape()[1])
    tmp = (numInput+numHidden)
    weights = tf.Variable(tf.random_normal([numInput, numHidden], 
                          stddev=tf.sqrt(1./tmp)), 
                          name = "w")
    bias = tf.Variable(tf.zeros( [numHidden,]), name="b")
    weights_loss = tf.reduce_sum(weights**2)/2.
    tf.add_to_collection("weights_loss", weights_loss)
    return tf.nn.bias_add(tf.matmul(x, weights), bias)

In [4]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=(None, 28,28))
y_target = tf.placeholder(tf.float32, [None,], name='target_y')
y_onehot = tf.to_float(tf.equal(tf.expand_dims(y_target, 1),
                                tf.to_float(tf.constant(
                                    np.arange(10).reshape(1,-1)))))
x_reshape = tf.reshape(x, [-1, 28*28])
h = x_reshape
for i in range(1):
    h = tf.nn.relu(linear(h, 1000))
yhat = (linear(h, 10))
crossEntropyError = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(yhat, y_onehot))
weightsError = tf.add_n(tf.get_collection("weights_loss"))
loss = crossEntropyError + 3e-4*weightsError
acc = tf.reduce_mean(tf.to_float(
                    tf.equal(tf.to_float(tf.arg_max(yhat, 1)), 
                             tf.to_float(y_target))))
train_op = tf.train.AdamOptimizer(learning_rate=0.001,).minimize(loss)

In [5]:
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()

In [9]:
sess.run(init)
loss.eval(feed_dict={x:validData, y_target:validTarget})
acc.eval(feed_dict={x:validData, y_target:validTarget})

0.079999998

In [10]:
B = 500

In [ ]:
for step in xrange(0,1000):
        randIdx = np.arange(len(trainData))
        np.random.shuffle(randIdx)
        for i in range(int(len(trainData)/B)):
            feeddict = {x: trainData[i*B:(i+1)*B], y_target: trainTarget[i*B:(i+1)*B], }
            _, err, Acc = sess.run([train_op, crossEntropyError,acc], feed_dict=feeddict)
        Val_err = loss.eval(feed_dict={x:validData, y_target:validTarget})
        Val_Acc = acc.eval(feed_dict={x:validData, y_target:validTarget})
        print("Iter: %3d, CE-train: %4.2f, CE-valid: %4.2f, Acc-train: %4.2f, Acc-valid: %4.4f"%(step, err, Val_err, Acc, Val_Acc))

Iter:   0, CE-train: 0.42, CE-valid: 0.49, Acc-train: 0.88, Acc-valid: 0.8930
Iter:   1, CE-train: 0.33, CE-valid: 0.45, Acc-train: 0.91, Acc-valid: 0.9060
Iter:   2, CE-train: 0.28, CE-valid: 0.43, Acc-train: 0.93, Acc-valid: 0.9110
Iter:   3, CE-train: 0.25, CE-valid: 0.40, Acc-train: 0.93, Acc-valid: 0.9100
Iter:   4, CE-train: 0.21, CE-valid: 0.39, Acc-train: 0.94, Acc-valid: 0.9210
Iter:   5, CE-train: 0.17, CE-valid: 0.40, Acc-train: 0.95, Acc-valid: 0.9230
Iter:   6, CE-train: 0.17, CE-valid: 0.39, Acc-train: 0.96, Acc-valid: 0.9270
Iter:   7, CE-train: 0.15, CE-valid: 0.42, Acc-train: 0.97, Acc-valid: 0.9240
Iter:   8, CE-train: 0.12, CE-valid: 0.39, Acc-train: 0.97, Acc-valid: 0.9310
Iter:   9, CE-train: 0.10, CE-valid: 0.40, Acc-train: 0.97, Acc-valid: 0.9300
Iter:  10, CE-train: 0.07, CE-valid: 0.40, Acc-train: 0.98, Acc-valid: 0.9340
Iter:  11, CE-train: 0.06, CE-valid: 0.41, Acc-train: 0.99, Acc-valid: 0.9370
Iter:  12, CE-train: 0.09, CE-valid: 0.42, Acc-train: 0.97, Acc-